# データマイニング Report3

+ 全体の流れ
    + NLTKの解説本の0章〜12章まで、計13個のHTMLファイルをダウンロードせよ。
    + BoWベースの特徴ベクトル（Level 1 もしくは Level 2）を生成せよ。
    + 共起行列ベースの特徴ベクトル（Level3）を生成せよ。
    + ラベル付き文書に対して分類タスク（Level4）を実行せよ。
+ Level 1: 文書ファイル毎に、``Bag-of-Words``で特徴ベクトルを生成せよ。
+ Level 2: ``BoW``に``TF-IDF``で重み調整した特徴ベクトルを生成せよ。
+ Level 3: 単語の``共起行列``から特徴ベクトルを生成せよ。
+ Level 4: ``文書分類``せよ。
+ オプション例
    + 相互情報量から``特徴ベクトル``を生成してみよう。
    + 共起行列に基づいた特徴ベクトル、もしくは相互特徴量に基づいた特徴ベクトルを``SVD``により``次元削減``してみよう。
    + SVDによる次元削減時に``2次元``とせよ。気になる単語1つを選び、上位10件と下位10件を2次元空間にマッピングせよ。マッピング結果、どのように散らばっているか観察し、想定とどのぐらい似通っているか考察してみよう。
    + ``日本語文書``について自然言語処理してみよう。

In [6]:
import os
import nltk
from nltk.tokenize import wordpunct_tokenize, sent_tokenize
import numpy as np
import glob
import scipy.spatial.distance as distance

+ collect_words_eng(): 英文書集合から単語コードブック作成
    

nltkのdownloadするべきmoudle

In [28]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/e175751/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/e175751/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /Users/e175751/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Bag-of-Words

## 文書集合からターム素性集合（コードブック）を作る

In [54]:

def collect_words_eng(docs):
    '''
    英文書集合から単語コードブック作成。
    シンプルに文書集合を予め決めうちした方式で処理する。
    必要に応じて指定できるようにしていた方が使い易いかも。

    :param docs(list): 1文書1文字列で保存。複数文書をリストとして並べたもの。
    :return (list): 文分割、単語分割、基本形、ストップワード除去した、ユニークな単語一覧。
    '''
    
    codebook = []
    stopwords = nltk.corpus.stopwords.words('english') 
    
    #stopwords.append('.')   # ピリオドを追加。
    #stopwords.append(',')   # カンマを追加。
    #stopwords.append('')    # 空文字を追加。
    
    symbol = ["'", '"', ':', ';', '.', ',', '-', '!', '?', "'s"]
    clean_frequency = nltk.FreqDist(w.lower() for w in docs if w.lower() not in stopwords + symbol)
    
    wnl = nltk.stem.wordnet.WordNetLemmatizer()
    
    for doc in docs:
        for sent in sent_tokenize(doc):
            for word in wordpunct_tokenize(sent):
                this_word = wnl.lemmatize(word.lower())
                if this_word not in codebook and this_word not in clean_frequency:
                    codebook.append(this_word)
    return codebook

In [47]:
docs3 = []
docs3.append("This is test.")
docs3.append("That is test too.")
docs3.append("There are so many many tests.")

``clean_frequencya``を使った場合
これにより、vector数が10個になる

In [55]:
codebook = collect_words_eng(docs3)
print('codebook = ',codebook)

codebook =  ['this', 'is', 'test', '.', 'that', 'too', 'there', 'are', 'so', 'many']


``stopwords``のままの場合
これにより、vector数が2個となる

In [36]:
codebook = collect_words_eng(docs3)
print('codebook = ',codebook)

codebook =  ['test', 'many']


## コードブックを素性とする文書ベクトルを作る (直接ベクトル生成)

In [56]:
def make_vectors_eng(docs, codebook):
    '''コードブックを素性とする文書ベクトルを作る（直接ベクトル生成）

    :param docs(list): 1文書1文字列で保存。複数文書をリストとして並べたもの。
    :param codebook(list): ユニークな単語一覧。
    :return (list): コードブックを元に、出現回数を特徴量とするベクトルを返す。
    '''
    vectors = []
    wnl = nltk.stem.wordnet.WordNetLemmatizer()
    for doc in docs:
        this_vector = []
        fdist = nltk.FreqDist()
        for sent in sent_tokenize(doc):
            for word in wordpunct_tokenize(sent):
                this_word = wnl.lemmatize(word.lower())
                fdist[this_word] += 1
        for word in codebook:
            this_vector.append(fdist[word])
        vectors.append(this_vector)
    return vectors


In [57]:
vectors = make_vectors_eng(docs3, codebook)
for index in range(len(docs3)):
    print('docs[{}] = {}'.format(index,docs3[index]))
    print('vectors[{}] = {}'.format(index,vectors[index]))
    print('----')

docs[0] = This is test.
vectors[0] = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
----
docs[1] = That is test too.
vectors[1] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
----
docs[2] = There are so many many tests.
vectors[2] = [0, 0, 1, 1, 0, 0, 1, 1, 1, 2]
----


## ユークリッド距離

In [58]:
def euclidean_distance(vectors):
    vectors = np.array(vectors)
    distances = []
    for i in range(len(vectors)):
        temp = []
        for j in range(len(vectors)):
            temp.append(np.linalg.norm(vectors[i] - vectors[j]))
        distances.append(temp)
    return distances

In [59]:
distances = euclidean_distance(vectors)
print('# euclidean_distance')
for index in range(len(distances)):
    print(distances[index])


# euclidean_distance
[0.0, 1.7320508075688772, 3.0]
[1.7320508075688772, 0.0, 3.1622776601683795]
[3.0, 3.1622776601683795, 0.0]


## コサイン類似度

In [60]:
def cosine_similarity(vectors):
    vectors = np.array(vectors)
    distances = []
    for i in range(len(vectors)):
        temp = []
        for j in range(len(vectors)):
            temp.append(distance.cosine(vectors[i], vectors[j]))
        distances.append(temp)
    return distances

In [61]:
similarities = cosine_similarity(vectors)
print('# cosine_similarity')
for index in range(len(similarities)):
    print(similarities[index])

# cosine_similarity
[0.0, 0.3291796067500631, 0.6666666666666667]
[0.3291796067500631, 0.0, 0.7018576030000281]
[0.6666666666666667, 0.7018576030000281, 0.0]


## それでは実際に文章を分類する

fileのpathを配列に格納する

In [65]:
List_Data_NL=[]
for i in range(1,14):
    List_Data_NL = glob.glob( "./data/*.html")

In [66]:
sentence = []
for l in List_Data_NL:
    with open(l) as f:
        r = f.read()
        sentence.append(r)

In [67]:
len(sentence)

13

### コードブック生成

In [68]:
codebook = collect_words_eng(sentence)
print('codebook = ',codebook)

codebook =  ['<!--', 'saved', 'from', 'url', '=(', '0035', ')', 'http', '://', 'www', '.', 'nltk', 'org', '/', 'book', 'ch00', 'html', '-->', '<', 'xmlns', '="', 'w3', '1999', 'xhtml', '"', 'xml', ':', 'lang', 'en', '"><', 'head', '><', 'meta', '-', 'equiv', 'content', 'type', 'text', ';', 'charset', '=', 'utf', '8', 'script', 'language', 'javascript', '">', 'function', 'astext', '(', 'node', '{', 'return', 'innerhtml', 'replace', '(/(<([^>]+)>)/', 'ig', ',"")', '(/&', 'gt', ';/', ',', '">")', 'lt', '"<")', 'quot', '\'"\')', 'amp', '"&");', '}', 'copy_notify', 'bar_color', 'data', '//', 'the', 'outer', 'box', 'relative', '+', 'inline', 'positioning', 'var', 'box1', 'document', 'createelement', '("', 'div', '");', 'style', 'position', '";', 'display', 'top', '2em', 'left', '1em', 'a', 'shadow', 'for', 'fun', 'absolute', '"-', '1', '3em', 'background', '"#', '404040', 'inner', 'box2', 'border', '1px', 'solid', '#', 'a0a0a0', '"-.', 'white', 'padding', '".', '4em', 'fontstyle', 'normal', 

### 文書ベクトル

In [69]:
vectors = make_vectors_eng(sentence, codebook)
for index in range(len(docs3)):
    print('docs[{}] = {}'.format(index,docs3[index]))
    print('vectors[{}] = {}'.format(index,vectors[index]))
    print('----')

docs[0] = This is test.
vectors[0] = [36, 1, 25, 4, 1, 1, 24, 28, 27, 6, 419, 55, 26, 51, 48, 4, 11, 36, 322, 1, 312, 1, 1, 1, 77, 1, 208, 2, 2, 74, 11, 85, 3, 342, 1, 5, 6, 31, 215, 1, 38, 1, 4, 2, 64, 3, 148, 13, 3, 60, 14, 104, 11, 2, 5, 1, 5, 1, 4, 4, 4, 295, 1, 1, 1, 1, 1, 2, 1, 97, 3, 2, 40, 5, 190, 1, 2, 3, 12, 3, 2, 16, 8, 7, 3, 7, 50, 6, 43, 4, 19, 2, 22, 7, 21, 7, 151, 8, 88, 1, 2, 2, 30, 4, 21, 6, 1, 1, 11, 67, 26, 48, 50, 1, 2, 2, 32, 1, 2, 1, 5, 1, 1, 1, 1, 70, 1, 2, 18, 1, 7, 7, 135, 2, 1, 50, 234, 1, 4, 2, 1, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 6, 17, 3, 2, 2, 5, 2, 14, 1, 13, 10, 2, 2, 3, 1, 5, 21, 2, 1, 2, 3, 4, 1, 12, 510, 2, 1, 2, 11, 2, 4, 2, 2, 2, 2, 1, 2, 1, 1, 8, 16, 30, 1, 62, 1, 1, 1, 1, 4, 10, 3, 3, 2, 3, 3, 11, 2, 3, 5, 1, 1, 1, 6, 4, 11, 8, 2, 1, 10, 13, 1, 3, 1, 1, 1, 29, 2, 2, 1, 1, 1, 282, 32, 1, 14, 1, 1, 2, 5, 7, 3, 3, 8, 7, 6, 2, 2, 2, 4, 14, 7, 1, 98, 1, 2, 7, 38, 2, 28, 24, 33, 2, 59, 3, 1, 3, 43, 1, 1, 1, 6, 3, 1, 1, 1, 5, 33, 1, 1, 1, 10, 3, 

### ユークリッド距離を求める

In [70]:
distances = euclidean_distance(vectors)
print('# euclidean_distance')
for index in range(len(distances)):
    print(distances[index])


# euclidean_distance
[0.0, 6679.858830843658, 5464.20259507277, 1141.4512692182702, 4453.808931689819, 9850.352531762506, 3375.864037546536, 4086.666979336584, 4245.508567886774, 622.1864672266667, 8523.743191814263, 6163.175155713166, 7319.142436105476]
[6679.858830843658, 0.0, 2305.6504505236694, 7574.700192086813, 2764.4616112364447, 3383.432428762247, 3668.9183692200077, 2856.5473915200496, 2782.855547814151, 7115.195991678655, 2166.710409814842, 1111.17505371566, 1179.951693926493]
[5464.20259507277, 2305.6504505236694, 0.0, 6314.304158020898, 2083.3372266630286, 4900.032754992562, 2853.513623587594, 2160.9055971976195, 1740.3881750919822, 5849.436725018914, 3760.7815677063722, 1764.569635916928, 2851.036653570066]
[1141.4512692182702, 7574.700192086813, 6314.304158020898, 0.0, 5405.782367058445, 10710.075816725108, 4308.880248974204, 4964.367834880893, 5169.950483321866, 700.4391479636186, 9400.089627232284, 7044.810217457955, 8204.946617742251]
[4453.808931689819, 2764.461611236

### コサイン類似度を求める

In [71]:
similarities = cosine_similarity(vectors)
print('# cosine_similarity')
for index in range(len(similarities)):
    print(similarities[index])

# cosine_similarity
[0.0, 0.17977087542170034, 0.2071564483033992, 0.34910262492529764, 0.12383533297034244, 0.21936147859644517, 0.12913954952576256, 0.17222093373830205, 0.14157483521111536, 0.09335328250143249, 0.20025570960970174, 0.18887530203651393, 0.18676683291388385]
[0.17977087542170034, 0.0, 0.03802333563765159, 0.4618771275336162, 0.03541993970159363, 0.010045722691319314, 0.04221583888071889, 0.017748817300154873, 0.024826396823463992, 0.2796987351413548, 0.00943408843174054, 0.008723668330534506, 0.00782112281901215]
[0.2071564483033992, 0.03802333563765159, 0.0, 0.4842570046814054, 0.050401157481438474, 0.034212077557935205, 0.07412350618742447, 0.04449219497791068, 0.025974141280270868, 0.27236343542871677, 0.03806339432864425, 0.028348221953597208, 0.042793555618599766]
[0.34910262492529764, 0.4618771275336162, 0.4842570046814054, 0.0, 0.39714049316640165, 0.5077253864619633, 0.4120265718189392, 0.4260362760261448, 0.4090928455795916, 0.423721935717291, 0.4740377116815

## それぞれのFiIeの関係性をコサイン類似度で確認する

In [91]:
for i in range(0,len(sentence)):
    for j in range(0,len(sentence)):
        list=[]
        if i < j:
            print(i,j)
            list.append(sentence[i])
            list.append(sentence[j])
            
        else:
            continue
        codebook = collect_words_eng(list)
        vectors = make_vectors_eng(list, codebook)
        similarities = cosine_similarity(vectors)
        print('# cosine_similarity')
        for index in range(len(similarities)):
            print(similarities[index])

0 1
# cosine_similarity
[0.0, 0.17977087542170045]
[0.17977087542170045, 0.0]
0 2
# cosine_similarity
[0.0, 0.2071564483033992]
[0.2071564483033992, 0.0]
0 3
# cosine_similarity
[0.0, 0.34910262492529776]
[0.34910262492529776, 0.0]
0 4
# cosine_similarity
[0.0, 0.12383533297034233]
[0.12383533297034233, 0.0]
0 5
# cosine_similarity
[0.0, 0.21936147859644506]
[0.21936147859644506, 0.0]
0 6
# cosine_similarity
[0.0, 0.12913954952576245]
[0.12913954952576245, 0.0]
0 7
# cosine_similarity
[0.0, 0.17222093373830194]
[0.17222093373830194, 0.0]
0 8
# cosine_similarity
[0.0, 0.14157483521111514]
[0.14157483521111514, 0.0]
0 9
# cosine_similarity
[0.0, 0.09335328250143238]
[0.09335328250143238, 0.0]
0 10
# cosine_similarity
[0.0, 0.20025570960970163]
[0.20025570960970163, 0.0]
0 11
# cosine_similarity
[0.0, 0.18887530203651393]
[0.18887530203651393, 0.0]
0 12
# cosine_similarity
[0.0, 0.18676683291388385]
[0.18676683291388385, 0.0]
1 2
# cosine_similarity
[0.0, 0.0380233356376517]
[0.0380233356